## Imports

In [1]:
import json
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from string import punctuation
import re
from keras.utils import to_categorical
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Bidirectional
from keras.layers import Embedding
# from keras.utils.np_utils import probas_to_classes
from sklearn import preprocessing
from keras.models import Sequential
from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
%matplotlib inline

from matplotlib import rcParams
from random import shuffle
# figure size in inches
rcParams['figure.figsize'] = 11.7,8.27

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/hitkul/anaconda3/envs/ps3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


## Loading Data

In [2]:
def load_data_from_json(filename):
    with open(filename,'r') as f:
        return json.load(f)

In [3]:
master_dict = load_data_from_json('dataset/task1_headline_ABSA_train.json')

In [4]:
master_dict

{'1': {'info': [{'aspects': "['Corporate/Appointment']",
    'sentiment_score': '-0.374',
    'snippets': "['set to step down']",
    'target': 'Royal Mail'}],
  'sentence': 'Royal Mail chairman Donald Brydon set to step down'},
 '100': {'info': [{'aspects': "['Market/Volatility/Volatility']",
    'sentiment_score': '-0.827',
    'snippets': "['down from record high']",
    'target': 'Weir'}],
  'sentence': 'Slump in Weir leads FTSE down from record high'},
 '1000': {'info': [{'aspects': "['Corporate/Regulatory']",
    'sentiment_score': '0.549',
    'snippets': "['FDA approval for key new lung cancer pill']",
    'target': 'AstraZeneca'}],
  'sentence': 'AstraZeneca wins FDA approval for key new lung cancer pill'},
 '1002': {'info': [{'aspects': "['Corporate/Stategy']",
    'sentiment_score': '-0.266',
    'snippets': "['cut 945 jobs as part of 3-year restructuring plan']",
    'target': 'Lloyds'}],
  'sentence': 'UPDATE 1-Lloyds to cut 945 jobs as part of 3-year restructuring plan'},

## Into Data

In [5]:
def extract_data_from_dict(master_dict):
    id_sentence = []
    sentence = []
    aspect = []
    target = []
    for key in master_dict.keys():
        id_sentence.append(int(key))
        sentence.append(master_dict[key]['sentence'])
        temp_target=[]
        temp_aspect = []
        for element in master_dict[key]['info']:
            temp_target.append(element['target'])
            temp_aspect.append(eval(element['aspects']))
        target.append(temp_target)
        aspect.append(temp_aspect)
    return id_sentence,sentence,aspect,target

In [6]:
id_sentence,sentence,aspect,target = extract_data_from_dict(master_dict)

In [10]:
target

[['Royal Mail'],
 ['AstraZeneca'],
 ['Morrisons'],
 ['Insight'],
 ['Primark'],
 ['Pearson'],
 ['TalkTalk', 'Tesco'],
 ['Unilever'],
 ['Tesco'],
 ['Royal Dutch Shell'],
 ['Shire'],
 ['China Merchants Group'],
 ['AstraZeneca', 'Teva'],
 ['EasyJet'],
 ['Blinkbox', 'Tesco'],
 ['Shire'],
 ['Ocwen'],
 ['Morrisons'],
 ['Blinkbox', 'Tesco'],
 ['Shire'],
 ['Omnis Pharmaceuticals'],
 ['Shire'],
 ['Thetrainline.com'],
 ['IAG'],
 ['Tesco'],
 ['Holcim Lafarge'],
 ['Persimmon'],
 ['BP'],
 ['Reed Elsevier'],
 ['Meggitt'],
 ['Weir'],
 ['Bunzl'],
 ['Lloyds'],
 ['Novartis'],
 ['Meggitt'],
 ['Astrazeneca'],
 ['ARM'],
 ['ConAgra'],
 ['BG Group'],
 ['Lonmin'],
 ['Schroders'],
 ['Tullow Oil'],
 ['Royal Dutch Shell'],
 ['Standard Chartered'],
 ['Standard Life'],
 ['Dixons Carphone', 'Tesco'],
 ['MillerCoors'],
 ['RBS'],
 ['HSBC'],
 ['Tesco'],
 ['Morrisons'],
 ['Persimmon'],
 ['Lafarge Holcim'],
 ['Tesco'],
 ['Deere', 'Exxon'],
 ['Aviva'],
 ['AstraZeneca'],
 ['Rolls-Royce'],
 ['Whitbread'],
 ['Berkshire Hatha